In [8]:
import numpy as np
import pandas as pd
import ast
import pywt
from scipy.signal import welch, spectrogram
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve,
                             average_precision_score)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# ----------------------------
# Data Loading and Preparation
# ----------------------------
df = pd.read_csv('final_classification_dataset.csv')

def safe_convert(signal):
    try:
        if isinstance(signal, str):
            arr = np.array(ast.literal_eval(signal))
        elif isinstance(signal, np.ndarray):
            arr = signal
        else:
            arr = np.zeros(100)
        if len(arr) < 100:
            arr = np.pad(arr, (0, 100 - len(arr)), 'constant')
        return arr[:100]
    except:
        return np.zeros(100)

df['clean_signal'] = df['clean_signal'].apply(safe_convert)
df['noise_signal'] = df['noise_signal'].apply(safe_convert)

# ----------------------------
# Feature Extraction Functions
# ----------------------------
def extract_enhanced_features(signal, fs=1000):
    features = {
        'mean': np.mean(signal),
        'std': np.std(signal),
        'max': np.max(signal),
        'min': np.min(signal),
        'median': np.median(signal),
        'q25': np.percentile(signal, 25),
        'q75': np.percentile(signal, 75),
        'rms': np.sqrt(np.mean(signal**2)),
        'zero_crossings': ((signal[:-1] * signal[1:]) < 0).sum(),
        'entropy': np.sum(-signal * np.log(signal + 1e-10)),
        'skewness': pd.Series(signal).skew(),
        'kurtosis': pd.Series(signal).kurtosis()
    }
    try:
        f, Pxx = welch(signal, fs=fs, nperseg=min(len(signal), 256))
        features.update({
            'spectral_centroid': np.sum(f * Pxx) / np.sum(Pxx),
            'spectral_bandwidth': np.sqrt(np.sum((f - features['spectral_centroid'])**2 * Pxx)),
            'spectral_entropy': -np.sum(Pxx * np.log2(Pxx + 1e-12)),
            'spectral_flatness': np.exp(np.mean(np.log(Pxx + 1e-10))) / np.mean(Pxx),
            'spectral_energy': np.sum(Pxx),
            'peak_frequency': f[np.argmax(Pxx)]
        })
    except:
        features.update({k: 0 for k in ['spectral_centroid', 'spectral_bandwidth', 
                                          'spectral_entropy', 'spectral_flatness',
                                          'spectral_energy', 'peak_frequency']})
    try:
        coeffs = pywt.wavedec(signal, 'db1', level=3)
        for i, c in enumerate(coeffs):
            features.update({
                f'wavelet_l{i}_mean': np.mean(c),
                f'wavelet_l{i}_std': np.std(c),
                f'wavelet_l{i}_energy': np.sum(c**2),
                f'wavelet_l{i}_entropy': -np.sum(c**2 * np.log(c**2 + 1e-10))
            })
        total_energy = np.sum([np.sum(c**2) for c in coeffs])
        features.update({
            'wavelet_total_energy': total_energy,
            'wavelet_energy_ratio': np.sum(coeffs[0]**2) / (total_energy + 1e-10)
        })
    except:
        wavelet_features = {f'wavelet_l{i}_{j}': 0 for i in range(4) for j in ['mean', 'std', 'energy', 'entropy']}
        wavelet_features.update({'wavelet_total_energy': 0, 'wavelet_energy_ratio': 0})
        features.update(wavelet_features)
    return features

def extract_spectrogram_features(signal, fs=1000):
    features = {}
    try:
        f, t, Sxx = spectrogram(signal, fs=fs, nperseg=256)
        spectral_centroid = np.sum(f[:, None] * Sxx, axis=0) / np.sum(Sxx, axis=0)
        features.update({
            'spec_centroid_mean': np.mean(spectral_centroid),
            'spec_centroid_std': np.std(spectral_centroid),
            'spec_centroid_max': np.max(spectral_centroid),
            'spec_centroid_min': np.min(spectral_centroid)
        })
        spectral_bandwidth = np.sqrt(np.sum((f[:, None] - spectral_centroid)**2 * Sxx, axis=0) / np.sum(Sxx, axis=0))
        features.update({
            'spec_bandwidth_mean': np.mean(spectral_bandwidth),
            'spec_bandwidth_std': np.std(spectral_bandwidth)
        })
        spectral_entropy = -np.sum(Sxx * np.log(Sxx + 1e-10), axis=0)
        features.update({
            'spec_entropy_mean': np.mean(spectral_entropy),
            'spec_entropy_std': np.std(spectral_entropy)
        })
        spectral_flatness = np.exp(np.mean(np.log(Sxx + 1e-10), axis=0)) / np.mean(Sxx, axis=0)
        features.update({
            'spec_flatness_mean': np.mean(spectral_flatness),
            'spec_flatness_std': np.std(spectral_flatness)
        })
        spectral_energy = np.sum(Sxx, axis=0)
        features.update({
            'spec_energy_mean': np.mean(spectral_energy),
            'spec_energy_std': np.std(spectral_energy),
            'spec_total_energy': np.sum(Sxx),
            'spec_peak_freq': f[np.argmax(np.mean(Sxx, axis=1))]
        })
    except:
        spec_features = {
            'spec_centroid_mean': 0, 'spec_centroid_std': 0,
            'spec_centroid_max': 0, 'spec_centroid_min': 0,
            'spec_bandwidth_mean': 0, 'spec_bandwidth_std': 0,
            'spec_entropy_mean': 0, 'spec_entropy_std': 0,
            'spec_flatness_mean': 0, 'spec_flatness_std': 0,
            'spec_energy_mean': 0, 'spec_energy_std': 0,
            'spec_total_energy': 0, 'spec_peak_freq': 0
        }
        features.update(spec_features)
    return features

def extract_all_features(signal, fs=1000):
    if isinstance(signal, str):
        try:
            signal = np.array(ast.literal_eval(signal))
        except:
            signal = np.zeros(100)
    features = {}
    features.update(extract_enhanced_features(signal, fs))
    features.update(extract_spectrogram_features(signal, fs))
    return features

# ----------------------------
# Extração dos Features
# ----------------------------
print("Extraindo features dos sinais...")
clean_features = pd.DataFrame(df['clean_signal'].apply(lambda x: extract_all_features(x, fs=1000)).tolist()).add_prefix('clean_')
noise_features = pd.DataFrame(df['noise_signal'].apply(lambda x: extract_all_features(x, fs=1000)).tolist()).add_prefix('noise_')
X = pd.concat([clean_features, noise_features, df[['clean_signal_strength', 'noise_signal_strength']]], axis=1)
y = df['class']
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

# ----------------------------
# Divisão em Treino e Teste
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# ----------------------------
# Balanceamento e Escalonamento
# ----------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# ----------------------------
# Construção e Treinamento do Modelo MLP
# ----------------------------
print("Construindo a rede neural MLP...")

model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Exemplo: ajuste de class_weight (opcional, se desejar penalizar mais a classe positiva)
# Como os dados já foram balanceados pelo SMOTE, esse parâmetro pode ser testado para melhorar a sensibilidade.
class_weight = {0: 1.0, 1: 1.5}

print("Treinando a rede neural...")
history = model.fit(X_train_scaled, y_train_res, epochs=100, batch_size=32, validation_split=0.3,
                    callbacks=[early_stop], verbose=2, class_weight=class_weight)

# ----------------------------
# Avaliação do Modelo MLP
# ----------------------------
print("Avaliando o modelo MLP...")
y_pred_prob = model.predict(X_test_scaled)
# Ajuste do threshold: experimente valores abaixo de 0.5 para melhorar o recall da classe positiva
threshold = 0.45
y_pred = (y_pred_prob > threshold).astype(int)

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Не сигнал', 'Сигнал присутствует']))

plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Не сигнал', 'Сигнал присутствует'],
            yticklabels=['Не сигнал', 'Сигнал присутствует'])
plt.title('Matriz de Confusão')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.show()

fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = roc_auc_score(y_test, y_pred_prob)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
ap_score = average_precision_score(y_test, y_pred_prob)
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f'PR (AP = {ap_score:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject